In [9]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\appunti\Project.toml`
  Resolving package versions...
  Installed LIBSVM ───── v0.5.0
  Installed libsvm_jll ─ v3.24.0+1
  Installed LIBLINEAR ── v0.5.1
Updating `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\appunti\Project.toml`
  [b1bec4e5] + LIBSVM v0.5.0
Updating `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\appunti\Manifest.toml`
  [2d691ee1] + LIBLINEAR v0.5.1
  [b1bec4e5] + LIBSVM v0.5.0
  [08558c22] + libsvm_jll v3.24.0+1
   Building LIBLINEAR → `C:\Users\i am\.julia\packages\LIBLINEAR\yTdp5\deps\build.log`


In [10]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

┌ Info: Precompiling LIBSVM [b1bec4e5-fd48-53fe-b0cb-9723c09d164b]
└ @ Base loading.jl:1278


In [25]:
#carichiamo e splittiamo dati in dati su cui allenare modello e dati su cui testare iul modello
iris = dataset("datasets", "iris")
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]


irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) # randsubset prende una struct, scorre gli elem e, ogni elem è ritornato con p probabilità

        push!(keepids,rowids...)
    end
    return keepids
end

trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids);

#ci serve una fun che ci dica quanto accurato è un modello
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)
#funzione ritorna rapporto var predette correttamente / tot var predette

findaccuracy (generic function with 1 method)

In [26]:
#fun prende val prdetti(continui) e ritorna classe(discreta)
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

In [27]:
#metodo 1 lasso
path = glmnet(X[trainids,:], y[trainids])#usiamo trainids per creare modello
cv = glmnetcv(X[trainids,:], y[trainids])#trova lambda migliore

mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda])

Least Squares GLMNet Solution Path (1 solutions for 4 predictors in 109 passes):
─────────────────────────────
     df   pct_dev           λ
─────────────────────────────
[1]   4  0.940337  0.00109283
─────────────────────────────

In [28]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)#applichiamo modello su testids

predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9761904761904762

In [29]:
#metodo 2 ridge
#uguale a prima ma con alpha=0
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9761904761904762

In [30]:
# metodo 3 rete elastica
#alpha a 0.5
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9761904761904762

In [31]:
#metodo 4 albero decisionale
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9047619047619048

In [32]:
#metodo 5 foreste casuali

model = RandomForestClassifier(n_trees=20)#dal pkg decisiontree
DecisionTree.fit!(model, X[trainids,:], y[trainids])
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9047619047619048

In [34]:
#metodo 6 vicino più vicino
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')
queries = X[testids,:]
idxs, dists = knn(kdtree, queries', 5, true)
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(argmax(DataFrame(possible_labels[i])[1,:]))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9523809523809523

In [35]:
#metodo 7 support vector machines
Xtrain = X[trainids,:]
ytrain = y[trainids]
model = svmtrain(Xtrain', ytrain)
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9761904761904762